In [2]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import csv
import math
import operator
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import os
import json
import asyncio
import requests
import aiohttp
from nltk.tokenize import RegexpTokenizer
tqdm.pandas()

### Full

In [3]:
data_path = '../data/'

In [4]:
dfproc = pd.read_csv(data_path + 'PROCEDURES_ICD.csv')
dfdiag = pd.read_csv(data_path + 'DIAGNOSES_ICD.csv')

In [4]:
dfproc.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [5]:
dfdiag.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


Put a period in the right place because the MIMIC-3 data files exclude them.Generally, procedure codes have dots after the first two digits, while diagnosis codes have dots after the first three digits.

In [5]:
def reformat(code, is_diag):
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

In [7]:
dfdiag['ICD9_CODE'] = dfdiag.apply(lambda row: str(reformat(str(row[4]), True)), axis=1)
dfproc['ICD9_CODE'] = dfproc.apply(lambda row: str(reformat(str(row[4]), False)), axis=1)

In [8]:
dfcodes = pd.concat([dfdiag, dfproc])
dfcodes.to_csv(data_path + 'ALL_CODES.csv', index=False,
               columns=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'],
               header=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'])


In [9]:
dfcodes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,403.01
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,582.81
3,1300,109,172335,4.0,585.5
4,1301,109,172335,5.0,425.4


In [10]:
df = pd.read_csv(data_path + 'ALL_CODES.csv',dtype={"ICD9_CODE": str})
len(df['ICD9_CODE'].unique())

8994

In [11]:
# df = df[df['ICD9_CODE'].notna()]

In [12]:
icd9codes = df['ICD9_CODE'].unique()
len(icd9codes)

8994

In [13]:
df.to_csv(data_path + 'ALL_CODES.csv', index=False)

### Get Discharge Summary

In [14]:
noteeventsdf = pd.read_csv(data_path + 'NOTEEVENTS.csv')

/tmp/ipykernel_2713912/772925661.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteeventsdf = pd.read_csv(data_path + 'NOTEEVENTS.csv')


In [15]:
noteeventsdf.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [16]:
len(noteeventsdf.index)

2083180

In [17]:
dich_full = noteeventsdf[noteeventsdf['CATEGORY']=='Discharge summary']

In [18]:
len(dich_full.index)

59652

In [19]:
len(dich_full['HADM_ID'].unique())

52726

In [20]:
# dich_full = dich_full[dich_full['HADM_ID'].notna()]

In [21]:
dich_full = dich_full.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [22]:
dich_full.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
48470,44005,3,145834.0,2101-10-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2101-10-20**] Discharg...
4782,4788,4,185777.0,2191-03-23,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-3-16**] Discharge...
24476,20825,6,107064.0,2175-06-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2175-5-30**] Dischar...
22764,20070,9,150750.0,2149-11-13,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2149-11-9**] Dischar...
57341,57115,9,150750.0,2149-11-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 10050**], [**Known fi..."


In [23]:
dich_full.to_csv(data_path + 'disch_full.csv', index=False)

In [24]:
del noteeventsdf

In [3]:
dich_full = pd.read_csv(data_path + 'disch_full.csv')
dich_full.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,44005,3,145834.0,2101-10-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2101-10-20**] Discharg...
1,4788,4,185777.0,2191-03-23,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2191-3-16**] Discharge...
2,20825,6,107064.0,2175-06-15,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2175-5-30**] Dischar...
3,20070,9,150750.0,2149-11-13,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2149-11-9**] Dischar...
4,57115,9,150750.0,2149-11-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 10050**], [**Known fi..."


### Sort

In [26]:
labeldf = pd.read_csv(data_path + 'ALL_CODES.csv',dtype={"ICD9_CODE": str})

In [27]:
len(labeldf['ICD9_CODE'].unique())

8994

In [28]:
labeldf = labeldf.drop(['ROW_ID'],axis=1)
labeldf = labeldf.sort_values(['SUBJECT_ID', 'HADM_ID'])
labeldf.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
608,2,163353,1.0,V30.01
609,2,163353,2.0,V05.3
610,2,163353,3.0,V29.0
746132,2,163353,1.0,99.55
611,3,145834,1.0,038.9


In [29]:
len(labeldf['ICD9_CODE'].unique())

8994

In [30]:
labeldf.to_csv(data_path + 'ALL_CODES.csv', index=False)

In [31]:
len(labeldf['HADM_ID'].unique())

58976

### Filter

In [4]:
labeldf = pd.read_csv(data_path + 'ALL_CODES.csv')
labeldf.head()

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,2,163353,1.0,V30.01
1,2,163353,2.0,V05.3
2,2,163353,3.0,V29.0
3,2,163353,1.0,99.55
4,3,145834,1.0,038.9


In [5]:
len(labeldf['ICD9_CODE'].unique())

8994

In [6]:
len(dich_full['HADM_ID'].unique()), len(labeldf['HADM_ID'].unique())

(52726, 58976)

In [7]:
admission_ids = list(dich_full['HADM_ID'].unique()) 
labeldf = labeldf.loc[labeldf['HADM_ID'].isin(admission_ids)]

In [8]:
len(labeldf['ICD9_CODE'].unique())

8908

In [18]:
labeldf = labeldf[labeldf['ICD9_CODE'].notna()]

In [19]:
labeldf.to_csv(data_path + 'ALL_CODES_filtered.csv', index=False)

In [20]:
labeldf = pd.read_csv(data_path + 'ALL_CODES_filtered.csv')
len(labeldf['HADM_ID'].unique())

52722

In [21]:
len(labeldf['ICD9_CODE'].unique())

8907

In [22]:
labeldf.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,3,145834,038.9
1,3,145834,785.59
2,3,145834,584.9
3,3,145834,427.5
4,3,145834,410.71


In [23]:
dich_full = pd.read_csv(data_path + 'disch_full.csv')
admission_ids = list(labeldf['HADM_ID'].unique()) 
dich_full = dich_full.loc[dich_full['HADM_ID'].isin(admission_ids)]

In [24]:
dich_full.to_csv(data_path + 'disch_full.csv', index=False)

In [25]:
dich_full = pd.read_csv(data_path + 'disch_full.csv')
len(dich_full['HADM_ID'].unique())

52722

### concatenate

In [26]:
labeldf = pd.read_csv(data_path + 'ALL_CODES_filtered.csv',dtype={"ICD9_CODE": str})

In [28]:
# labeldf = labeldf.drop(['SEQ_NUM'], axis=1)

In [29]:
labeldf.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,3,145834,038.9
1,3,145834,785.59
2,3,145834,584.9
3,3,145834,427.5
4,3,145834,410.71


In [30]:
labeldf=labeldf.groupby(['SUBJECT_ID','HADM_ID']).aggregate([('ICD9_CODE',' '.join)])

In [32]:
labeldf = labeldf.droplevel(axis=1, level=1).reset_index()

In [35]:
labeldf.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,3,145834,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [36]:
dich_full = pd.read_csv(data_path + 'disch_full.csv')

In [37]:
dich_full=dich_full.groupby(['SUBJECT_ID','HADM_ID']).aggregate([('TEXT',' '.join)])

/tmp/ipykernel_2718364/1083673685.py:1: FutureWarning: ['ROW_ID', 'CHARTTIME', 'STORETIME', 'CGID', 'ISERROR'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  dich_full=dich_full.groupby(['SUBJECT_ID','HADM_ID']).aggregate([('TEXT',' '.join)])


In [38]:
dich_full = dich_full.droplevel(axis=1, level=1).reset_index()
dich_full = dich_full.drop(['CHARTDATE','CATEGORY','DESCRIPTION'],axis=1)

In [39]:
dich_full.head()

,SUBJECT_ID,HADM_ID,TEXT
0,3,145834.0,Admission Date: [**2101-10-20**] Discharg...
1,4,185777.0,Admission Date: [**2191-3-16**] Discharge...
2,6,107064.0,Admission Date: [**2175-5-30**] Dischar...
3,9,150750.0,Admission Date: [**2149-11-9**] Dischar...
4,10,184167.0,Admission Date: [**2103-6-28**] Dischar...


In [40]:
len(dich_full.index)

52722

In [41]:
notes_labeled_full = pd.merge(labeldf,dich_full,on=['HADM_ID','SUBJECT_ID'],how='inner')

In [43]:
notes_labeled_full = notes_labeled_full.loc[:,['SUBJECT_ID','HADM_ID','TEXT','ICD9_CODE']]

In [44]:
notes_labeled_full.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,3,145834,Admission Date: [**2101-10-20**] Discharg...,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,Admission Date: [**2191-3-16**] Discharge...,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,Admission Date: [**2175-5-30**] Dischar...,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,Admission Date: [**2149-11-9**] Dischar...,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,Admission Date: [**2103-6-28**] Dischar...,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [45]:
len(notes_labeled_full.index)

52722

In [46]:
notes_labeled_full.to_csv(data_path + 'notes_labeled.csv', index=False)

In [47]:
del dich_full
del notes_labeled_full

### Preprocess Text

In [48]:
df  = pd.read_csv(data_path + 'notes_labeled.csv')

In [49]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,3,145834,Admission Date: [**2101-10-20**] Discharg...,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,Admission Date: [**2191-3-16**] Discharge...,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,Admission Date: [**2175-5-30**] Dischar...,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,Admission Date: [**2149-11-9**] Dischar...,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,Admission Date: [**2103-6-28**] Dischar...,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [50]:
df.iloc[0,2]

"Admission Date:  [**2101-10-20**]     Discharge Date:  [**2101-10-31**]\n\nDate of Birth:   [**2025-4-11**]     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**]\nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn [**2101-10-20**], he presented again to the [**Hospital1 346**] after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  I

In [51]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [52]:
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(discharge_summary):
#     ds = discharge_summary
#     ds = re.sub('\[.*\]', '', ds)
#     ds = ds.replace('\n',' ')
#     ds = ds.replace('\r',' ')
#     terms = ['Admission Date:','Discharge Date:','Date of Birth:','Service:','ADDENDUM:','Date/Time:','Completed by:','Dictated By:',' D:',' T:','JOB#:','Followup Instructions:','Follow up w/ Dr.', 'Provider:']
#     for term in terms:
#         ds = re.sub(term,'',ds)
#     ds = re.sub('\d+:\d+','',ds)
#     ds = re.sub(' +',' ',ds)
#     ds = re.sub('\.','',ds)
#     ds = ds.strip()
#     tokens = [t.lower() for t in tokenizer.tokenize(ds)]
#     ds = ' '.join(tokens)
#     return ds

    ds = discharge_summary
    ds = re.sub('\[.*\]', '', ds)
    ds = ds.replace('\n',' ')
    ds = ds.replace('\r',' ')
#     terms = ['Admission Date:','Discharge Date:','Date of Birth:','Service:','ADDENDUM:','Date/Time:','Completed by:','Dictated By:',' D:',' T:','JOB#:','Followup Instructions:','Follow up w/ Dr.', 'Provider:']
#     for term in terms:
#         ds = re.sub(term,'',ds)
    ds = re.sub('\d+:\d+','',ds)
    ds = re.sub('\d+','',ds)
    ds = re.sub(' +',' ',ds)
    ds = re.sub('\.','',ds)
    ds = ds.strip()
    tokens = [t.lower() for t in tokenizer.tokenize(ds)]
    tokens = [w for w in tokens if not w in stop_words]
    ds = ' '.join(tokens)   
    return ds
preprocess_text(df.iloc[0,2])

'admission date date birth sex service medicine chief complaint admitted rehabilitation hypotension systolic blood pressure decreased urine output history present illness patient year old male hospitalized undergoing left femoral bypass graft subsequently discharged rehabilitation facility found systolic blood pressure urine output hours foley catheter placed rehabilitation facility yielded cc murky brown urine may also purulent discharge penile meatus time presentation emergency department patient without subjective complaints emergency department found systolic blood pressure given liters intravenous fluids transiently started dopamine systolic blood pressure past medical history coronary artery disease diffuse vessel disease right dominant status post proximal left circumflex stent occlusion distal left circumflex status post right coronary artery stent percutaneous coronary intervention diagonal left circumflex small proximal left anterior descending artery small distal left anteri

In [53]:
df['TEXT'] = df.apply(lambda row: str(preprocess_text(row[2])), axis=1)

In [54]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,3,145834,admission date date birth sex service medicine...,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,admission date date birth sex f service chief ...,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,admission date date birth sex f service admiss...,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,admission date date birth sex service neurolog...,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,admission date date birth sex f service histor...,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [56]:
types = set()
num_tok = 0
for row in df.itertuples():
    for w in row[-2].split(' '):
        types.add(w)
        num_tok += 1
print("Num types", len(types))
print("Num tokens", str(num_tok))

Num types 139263
Num tokens 51895155


In [57]:
df.to_csv(data_path + 'processed_notes_labeled.csv', index=False)

In [58]:
del df

# Full

In [57]:
df = pd.read_csv(data_path + 'processed_notes_labeled.csv')

In [58]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,3,145834,admission date date birth sex service medicine...,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,admission date date birth sex f service chief ...,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,admission date date birth sex f service admiss...,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,admission date date birth sex service neurolog...,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,admission date date birth sex f service histor...,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [59]:
len(df.index)

52722

In [60]:
# len(df['ICD9_CODE'].unique())

In [61]:
labeldf = pd.read_csv(data_path + 'ALL_CODES_filtered.csv',dtype={"ICD9_CODE": str})

In [62]:
labeldf.head()

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,3,145834,038.9
1,3,145834,785.59
2,3,145834,584.9
3,3,145834,427.5
4,3,145834,410.71


In [63]:
full_codes = labeldf['ICD9_CODE'].unique()
len(full_codes)

8907

In [64]:
train_ids = pd.read_csv(data_path+'train_full_hadm_ids.csv',names=['HADM_ID'],header=None)
train_admisn_ids  = list(train_ids['HADM_ID'])
train_full = df.loc[df['HADM_ID'].isin(train_admisn_ids)]

In [65]:
train_full.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
0,3,145834,admission date date birth sex service medicine...,038.9 785.59 584.9 427.5 410.71 428.0 682.6 42...
1,4,185777,admission date date birth sex f service chief ...,042 136.3 799.4 276.3 790.7 571.5 041.11 V09.0...
2,6,107064,admission date date birth sex f service admiss...,403.91 444.0 997.2 276.6 276.7 285.9 275.3 V15...
3,9,150750,admission date date birth sex service neurolog...,431 507.0 428.0 584.9 276.5 401.9 96.72 96.04
4,10,184167,admission date date birth sex f service histor...,V30.00 774.2 765.25 765.15 V29.0 99.83 99.15 96.6


In [66]:
len(train_full.index)

47719

In [67]:
train_full.to_csv(data_path + 'train_full.csv', index=False)

In [68]:
test_ids = pd.read_csv(data_path+'test_full_hadm_ids.csv',names=['HADM_ID'],header=None)
test_admisn_ids  = list(test_ids['HADM_ID'])
test_full = df.loc[df['HADM_ID'].isin(test_admisn_ids)]

In [69]:
test_full.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
49350,89109,144591,admission date date birth sex f service medici...,487.8 482.42 428.21 493.22 518.81 997.31 357.0...
49351,89112,183380,admission date date birth sex f service medici...,411.0 401.9 305.1 414.01 780.79 427.31 515 V45...
49352,89119,180906,admission date date birth sex service medicine...,790.7 585.6 507.0 999.31 428.22 403.91 112.2 4...
49353,89119,187268,admission date date birth sex service medicine...,038.43 785.52 585.6 599.0 403.91 428.22 008.45...
49354,89119,191630,admission date date birth sex service medicine...,599.0 585.6 403.91 560.0 428.22 428.0 707.03 7...


In [70]:
len(test_full.index)

3372

In [71]:
test_full.to_csv(data_path + 'test_full.csv', index=False)

In [72]:
dev_ids = pd.read_csv(data_path+'dev_full_hadm_ids.csv',names=['HADM_ID'],header=None)
dev_admisn_ids  = list(dev_ids['HADM_ID'])
dev_full = df.loc[df['HADM_ID'].isin(dev_admisn_ids)]

In [73]:
dev_full.head()

,SUBJECT_ID,HADM_ID,TEXT,ICD9_CODE
47719,83537,161562,admission date date birth sex service medicine...,410.71 578.0 285.1 401.9 272.4 V10.46 564.1 59...
47720,83542,103451,admission date date birth sex service allergie...,410.41 414.01 401.9 250.00 66. 36.07 45. 40. 3...
47721,83543,189598,admission date date birth sex service surgery ...,800.22 820.09 801.02 E826.1 E849.8 79.35
47722,83547,103269,admission date date birth sex f service cardio...,394.2 285.1 401.9 428.0 276.7 250.00 327.23 30...
47723,83550,116832,admission date date birth sex f service surger...,560.2 491.21 997.1 427.32 514 276.6 568.0 272....


In [74]:
len(dev_full.index)

1631

In [75]:
dev_full.to_csv(data_path + 'dev_full.csv', index=False)

In [76]:
ccodes = set()
for lcodes in train_full['ICD9_CODE'].unique():
    for code in lcodes.split(' '):
        ccodes.add(code)
        
for lcodes in test_full['ICD9_CODE'].unique():
    for code in lcodes.split(' '):
        ccodes.add(code)
        
for lcodes in dev_full['ICD9_CODE'].unique():
    for code in lcodes.split(' '):
        ccodes.add(code)
len(ccodes)

8907

In [77]:
diag_desc = pd.read_csv(data_path + 'D_ICD_DIAGNOSES.csv')
proc_desc = pd.read_csv(data_path + 'D_ICD_PROCEDURES.csv')

In [78]:
diag_desc['ICD9_CODE'] = diag_desc.apply(lambda row: str(reformat(str(row[1]), True)), axis=1)
proc_desc['ICD9_CODE'] = proc_desc.apply(lambda row: str(reformat(str(row[1]), False)), axis=1)

In [79]:
code_desc = pd.concat([diag_desc, proc_desc])
code_desc.to_csv(data_path + 'D_ICD.csv', index=False,
               columns=['ICD9_CODE','LONG_TITLE'],
               header=['ICD9_CODE','DESC'])

In [80]:
code_desc = pd.read_csv(data_path + 'D_ICD.csv',dtype={"ICD9_CODE": str})
code_desc

,ICD9_CODE,DESC
0,011.66,"Tuberculous pneumonia [any form], tubercle bac..."
1,011.70,"Tuberculous pneumothorax, unspecified"
2,011.71,"Tuberculous pneumothorax, bacteriological or h..."
3,011.72,"Tuberculous pneumothorax, bacteriological or h..."
4,011.73,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...
18444,99.59,Other vaccination and inoculation
18445,99.60,"Cardiopulmonary resuscitation, not otherwise s..."
18446,99.61,Atrial cardioversion
18447,99.62,Other electric countershock of heart


In [81]:
len(code_desc['ICD9_CODE'].unique())

18376

In [82]:
code_desc_full = code_desc.loc[code_desc['ICD9_CODE'].isin(full_codes)]

In [83]:
code_desc_full

,ICD9_CODE,DESC
14,011.86,"Other specified pulmonary tuberculosis, tuberc..."
15,011.90,"Pulmonary tuberculosis, unspecified, unspecified"
18,011.93,"Pulmonary tuberculosis, unspecified, tubercle ..."
19,011.94,"Pulmonary tuberculosis, unspecified, tubercle ..."
27,012.05,"Tuberculous pleurisy, tubercle bacilli not fou..."
...,...,...
18444,99.59,Other vaccination and inoculation
18445,99.60,"Cardiopulmonary resuscitation, not otherwise s..."
18446,99.61,Atrial cardioversion
18447,99.62,Other electric countershock of heart


In [84]:
fcodes = code_desc_full['ICD9_CODE'].unique()

In [85]:
len(fcodes)

8751

In [86]:
ncodes = []
for code in ccodes:
    if code not in fcodes:
        ncodes.append(code)

In [87]:
len(ncodes)

156

In [88]:
code_desc_full.to_csv(data_path + 'D_ICD_full.csv', index=False)

In [89]:
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_desc(desc):
    ds = desc
    ds = ds.replace('\n',' ')
    ds = ds.replace('\r',' ')
    ds = re.sub(' +',' ',ds)
    ds = re.sub('{\(,\),\[,\]}','',ds)
    ds = ds.strip()
    tokens = [t.lower() for t in tokenizer.tokenize(ds)]
    ds = ' '.join(tokens)
    return ds
preprocess_desc(code_desc_full.iloc[39,-1])

'malaria unspecified'

In [90]:
code_desc_full['DESC'] = code_desc_full.apply(lambda row: str(preprocess_desc(row[-1])), axis=1)

/var/tmp/pbs.14325.master/ipykernel_3353888/165692273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_desc_full['DESC'] = code_desc_full.apply(lambda row: str(preprocess_desc(row[-1])), axis=1)


In [91]:
# code_desc_full = code_desc_full.drop(['ROW_ID','SHORT_TITLE','LONG_TITLE'], axis=1)

In [92]:
code_desc_full

,ICD9_CODE,DESC
14,011.86,other specified pulmonary tuberculosis tubercl...
15,011.90,pulmonary tuberculosis unspecified unspecified
18,011.93,pulmonary tuberculosis unspecified tubercle ba...
19,011.94,pulmonary tuberculosis unspecified tubercle ba...
27,012.05,tuberculous pleurisy tubercle bacilli not foun...
...,...,...
18444,99.59,other vaccination and inoculation
18445,99.60,cardiopulmonary resuscitation not otherwise sp...
18446,99.61,atrial cardioversion
18447,99.62,other electric countershock of heart


In [93]:
for code in ncodes:
    df = {'ICD9_CODE':code, 'DESC':' '}
    code_desc_full = code_desc_full.append(df, ignore_index = True)

/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_de

/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_desc_full.append(df, ignore_index = True)
/var/tmp/pbs.14325.master/ipykernel_3353888/1976585064.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  code_desc_full = code_de

In [94]:
len(code_desc_full['ICD9_CODE'].unique())

8907

In [95]:
len(code_desc_full.index)

8968

In [96]:
code_desc_full = code_desc_full.drop_duplicates(subset=['ICD9_CODE'], keep='first')

In [97]:
code_desc_full.to_csv(data_path + 'D_ICD_full.csv', index=False)

In [98]:
code_desc_full.dtypes

ICD9_CODE    object
DESC         object
dtype: object

In [99]:
code_desc_full['ICD9_CODE'] = code_desc_full['ICD9_CODE'].astype(str)

In [100]:
code_desc_full['DESC'] = code_desc_full['DESC'].astype(str)

In [101]:
code_desc_full

,ICD9_CODE,DESC
0,011.86,other specified pulmonary tuberculosis tubercl...
1,011.90,pulmonary tuberculosis unspecified unspecified
2,011.93,pulmonary tuberculosis unspecified tubercle ba...
3,011.94,pulmonary tuberculosis unspecified tubercle ba...
4,012.05,tuberculous pleurisy tubercle bacilli not foun...
...,...,...
8963,523.3,
8964,337.0,
8965,608.2,
8966,173.2,
